### Correlation based Distance

상관계수는 선형적 상호의존성의 유용한 척도다. 일단 상관계수 행렬의 잡음과 주음이 제거되면 시스템에 대한 중요한 구조 정보를 보일 수 있다. 예를 들어서, 상호 연관성이 매우 높은 유가증권의 군집을 식별하고자 상관계수를 사용할 수 있다. 그러나 상관계수를 사용하기 전에 기술적 문제를 해결해야 한다. 상관계수는 비음성과 삼각 부등식 조건을 충족시키지 못하기 때문에 척도가 되지 못한다. 거리 척도는 집합에 대한 직관적인 위상관계를 유도하기 때문에 중요하다. 이러한 직관적 위상관계 없이 상호 의존성의 비거리적 측정을 비교하는 것은 다소 일관성 없는 결과를 초래할 수 있다. 예를 들어서 상관계수 (0.9, 1.0)의 차이는 (0.1, 0.2)의 차이와 같다. 물론 전자가 상호의존성의 관점에서 더 큰 차이를 가질 것이다.

크기 $T$의 랜덤 벡터 $X$와 $Y$와 상관계수 추정치 $\rho[X,Y]$를 고려하자. 상관계수는 단지 $\sigma_{X,Y} = \rho(X, Y)\sigma_X\sigma_Y$를 만족하면 되는데 여기서 $\sigma_{X,Y}$는 두 벡터의 공분산이고, $\sigma$는 표준 편차다. pearson correlation은 이들 조건을 만족하는 여러 상관계수 추정치 중 하나다. 이때, $d_\rho(X,Y) = \sqrt{\frac{1}{2(1 - \rho(X, Y))}}$는 거리 척도다.

### Marginal & Joint Entropy

상관계수 개념은 세 가지 중요한 결점을 제공한다. 첫째, 이는 두 랜덤 변수 간의 선형 상호의존성을 계량화한다. 이는 비선형관계를 무시한다. 둘째, 상관계수는 특이치에 크게 영향을 받는다. 셋째, 다변수 정규 분포의 경우를 넘어서는 응용은 의심스럽다. 우리는 어떤 두 실수 변수 간의 상관관계를 포착하고 싶지만, 두 변수가 이변수 정규분포를 따르지 않는다면 상관계수는 일반적으로 의미가 없어진다. 이러한 결함을 극복하고자 몇가지 정보이론 개념을 도입해야 한다.

$X$를 확률 $p(x)$로 집합 $S_X$로부터 $x$값을 취하는 이산 랜덤 변수라 하자. $X$의 엔트로피는 다음과 같이 정의된다.

$$H(X) = - \sum_{x \in S_X} p(x)\log(p(x))$$